In [ ]:
!pip install bitarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 7.7 MB/s 


In [ ]:
import sys
from bitarray import bitarray
import struct
from math import floor, sin

In [ ]:
# define the binary left rotation function
rot_left = lambda x, n: (x << n) | (x >> (256 - n))


# define the modular addition (mod 2^32)
mod_add = lambda x, y: (x + y) % (2**32)

**Hasher**


1-Ajustement de la taille pour qu'elle soit un multiple de 256

2- division par morceaux de 128

3- faire des opérations simples avec les mots H1 H2 H3 H4

4- faire le XOR avec les morceaux de 128




In [128]:
def Hash(s: str):
  s_bits = bitarray(endian='big')
  s_bits.frombytes(s.encode('utf-8'))
   # pad the string's bits with a 1 and then 0s until their length corresponds to 448 mod 512
  s_bits.append(1)
  while len(s_bits) % 128 != 0:
    s_bits.append(0)

  # convert back to little endian (since no more appending)
  s_bits = bitarray(s_bits, endian='little')

  # divide the message into 512 bits chunks
  chunks = [s_bits[i*128:(i+1)*128] for i in range(int(len(s_bits) /128))]

  for chunk in chunks:
    # divide the chunk into 32 bits long sub-chunks and convert back to int
    X = [chunk[i*32:(i+1)*32] for i in range(int(len(chunk)/32))]
    X = [int.from_bytes(w.tobytes(), byteorder='little') for w in X]
    W=[0,0,0,0]    
    H0 = 0x01234567
    H1 = 0x89abcdef
    H2 = 0xfedcba98
    H3 = 0x76543210

    for i in range(0,4):
      X[i]=X[i]^W[i] #XOR with the previous chank   
    
    A=mod_add(X[0],H0)
    rot_left(W[0],5)
    
    B=X[1] & H1
    rot_left(W[1],10)

    C=X[2] | H2
    rot_left(W[2],8)
  
    D=X[3] ^ H3
    rot_left(W[0],15)


    W[0]=mod_add(mod_add(A,B),D)
    W[1]=mod_add(D,C)
    W[2]=A^B^C
    W[3]=C^B|D



   # last step: convert the computed buffers to little endian
    
    return f"{format(W[0], '08x')}{format(W[1], '08x')}{format(W[2], '08x')}{format(W[3], '08x')}"



In [129]:
s='hello world'

X=Hash(s)
X

'41afbd4e7552f1f2490d51457e54bb3c'